In [ ]:
import mxnet as mx
import logging
import os
from gluoncv.utils import viz

In [ ]:
mx.random.seed(42)

In [ ]:
batch_size = 10

In [ ]:
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [ ]:
train_iter = mx.io.ImageRecordIter (
  path_imgrec = os.path.join(r'C:\Users\konst\mxnet\image_data\train_data', 'train_image_128x128.rec'),
  path_imgidx = os.path.join(r'C:\Users\konst\mxnet\image_data\train_data', 'train_image_128x128.idx'),  
  data_shape = (3, 128, 128),
  batch_size = batch_size,
)

In [ ]:
test_iter = mx.io.ImageRecordIter (
  path_imgrec = os.path.join(r'C:\Users\konst\mxnet\image_data\test_data', 'test_image_128x128.rec'),
  path_imgidx = os.path.join(r'C:\Users\konst\mxnet\image_data\test_data', 'test_image_128x128.idx'),  
  data_shape = (3, 128, 128),
  batch_size = batch_size,
)

In [ ]:
data = mx.sym.var('data')
data = mx.sym.flatten(data = data)

In [ ]:
for batch in test_iter:
    viz.plot_image(mx.nd.transpose(batch.data[0][0], (1, 2, 0)))
    break

In [ ]:
data = mx.sym.var('data')

#первый свёрточный
conv1 = mx.sym.Convolution(data = data, kernel = (5, 5), num_filter = 96, pad = (2, 2), stride = (2, 2))
tanh1 = mx.sym.Activation(data = conv1, act_type = "tanh")
pool1 = mx.sym.Pooling(data = tanh1, pool_type = "max", kernel = (3, 3), stride = (2, 2))

#второй свёрточный
conv2 = mx.sym.Convolution(data = pool1, kernel = (5, 5), num_filter = 256, pad = (2, 2), stride = (2, 2))
tanh2 = mx.sym.Activation(data = conv2, act_type = "tanh")
pool2 = mx.sym.Pooling(data = tanh2, pool_type = "max", kernel = (3, 3), stride = (2, 2))

#3 свёрточный
conv3 = mx.sym.Convolution(data = pool2, kernel = (3, 3), num_filter = 384, pad = (1, 1))
tanh3 = mx.sym.Activation(data = conv3, act_type = "tanh")
pool3 = mx.sym.Pooling(data = tanh3, pool_type = "max", kernel = (3, 3), stride = (2, 2))

#4 свёрточный
conv4 = mx.sym.Convolution(data = pool3, kernel = (3, 3), num_filter = 384, pad = (1, 1))
tanh4 = mx.sym.Activation(data = conv4, act_type = "tanh")
pool4 = mx.sym.Pooling(data = tanh4, pool_type = "max", kernel = (3, 3), stride = (2, 2))

#5 свёрточный
conv5 = mx.sym.Convolution(data = pool4, kernel = (3, 3), num_filter = 256, pad = (2, 2))
tanh5 = mx.sym.Activation(data = conv5, act_type = "tanh")
pool5 = mx.sym.Pooling(data = tanh5, pool_type = "max", kernel = (3, 3), stride = (2, 2))

#первый
flatten = mx.sym.flatten(data = pool5) 
fc1 = mx.symbol.FullyConnected(data = flatten, num_hidden = 4096)
tanh5 = mx.sym.Activation(data = fc1, act_type = "tanh")

fc2 = mx.symbol.FullyConnected(data = tanh5, num_hidden = 1024)
tanh6 = mx.sym.Activation(data = fc2, act_type = "tanh")

#выход
fc3 = mx.sym.FullyConnected(data = tanh6, num_hidden = 5)
lenet = mx.sym.SoftmaxOutput(data = fc3, name = 'softmax')

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)


lenet_model = mx.mod.Module(symbol = lenet, context = mx.gpu())

lenet_model.fit(train_iter,
                eval_data = test_iter,
                optimizer = 'sgd',
                optimizer_params = {'learning_rate':0.1},
                eval_metric = 'acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch = 5)

In [ ]:
mx.viz.plot_network(lenet)